In [ ]:
library(MAESTRO)
library(chromVAR)
library(Seurat)
library(Matrix)
# library(BSgenome.Hsapiens.UCSC.hg19)
# library(BSgenome.Hsapiens.UCSC.hg38)
library(BiocParallel)
library(motifmatchr)
library(SummarizedExperiment)
library(rhdf5)
library(dplyr)
library(motifmatchr)
# library(chromVARmotifs)
# library(JASPAR2016)

library(ggplot2)
# library(spam)
# library(VennDiagram)
library(stringr)
# library(ggrepel)
# library(ggsignif)
library(qs)
register(MulticoreParam(8))

In [ ]:
destination = 'example/PBMC/data/ATAC/filtered_mtx'

# PBMC

In [ ]:
# pbmc_RNA_object <- readRDS('example/PBMC/data/PBMC_granulocyte_sorted_10k_scRNA_Object.rds')

## ATAC-seq analysis

In [ ]:
pbmc_inputMat <- Read10X_h5('example/PBMC/data/ATAC/filtered_mtx/PBMC_granulocyte_sorted_10k_filtered_peak_count.h5')

In [ ]:
pbmc_inputMat

In [ ]:
# rna_bc <- rownames(pbmc_RNA_object$RNA@meta.data)

# key = read.table('example/PBMC/barcode_key.txt',sep='\t', header=TRUE)

# key[which(key$RNA %in% rna_bc),'ATAC']

# atac_bc <- key[which(key$RNA %in% rna_bc),'ATAC']

In [ ]:
# atac_match_bc <- colnames(pbmc_inputMat)[colnames(pbmc_inputMat) %in% atac_bc]

In [ ]:
# filter_pbmc_count <- pbmc_inputMat[,atac_match_bc]

In [ ]:
# gene.id = rownames(filter_pbmc_count)
# write10xCounts(
#   'example/PBMC/data/PBMC_granulocyte_sorted_10k_peak_count_filter_by_match.h5',
#   filter_pbmc_count,
#   barcodes = colnames(filter_pbmc_count),
#   gene.id = rownames(filter_pbmc_count),
#   gene.symbol = gene.id,
#   gene.type = "Chromatin Accessibility",
#   overwrite = FALSE,
#   type = "HDF5",
#   genome = "hg38",
#   version = "3",
#   chemistry = "Single Cell 3' v3",
#   original.gem.groups = 1L,
#   library.ids = "custom"
# )

In [ ]:
# pbmc.gene <- ATACCalculateGenescore(filter_pbmc_count, organism = "GRCh38")

In [ ]:
pbmc.ATAC.res <- ATACRunSeurat(inputMat = pbmc_inputMat,
                                 project = "atac",
                                 min.c = 50,
                                 min.p = 500,
                                 method = "LSI",
                                 dims.use = 1:30,
                                 cluster.res = 0.6,
                                 only.pos = TRUE,
                                 peaks.test.use = "presto",
                                 peaks.cutoff = 1e-05,
                                 peaks.pct = 0.1,
                                 peaks.logfc = 0.2,
                                 outdir = "example/PBMC/analysis/")

In [ ]:
# pbmc.ATAC.res$ATAC <- ATACAttachGenescore(ATAC = pbmc.ATAC.res$ATAC, RPmatrix = pbmc.gene)
# data(human.immune.CIBERSORT)
# pbmc.ATAC.res$ATAC <- ATACAnnotateCelltype(ATAC = pbmc.ATAC.res$ATAC,
#                                              signatures = human.immune.CIBERSORT,
#                                              min.score = 0.1,
#                                              genes.test.use = "presto",
#                                              genes.cutoff = 1E-5)

In [ ]:
DimPlot(pbmc.ATAC.res$ATAC, label = TRUE, pt.size = 0.2, repel = T)

In [ ]:
# pbmc.ATAC.res$ATAC@reductions$umap@cell.embeddings

In [ ]:
# DimPlot(pbmc.ATAC.res$ATAC, label = TRUE, pt.size = 0.2, repel = T, group.by='assign.ident')

In [ ]:
pbmc.ATAC.singlecellexperiment <- as.SingleCellExperiment(pbmc.ATAC.res$ATAC)

In [ ]:
saveRDS(pbmc.ATAC.singlecellexperiment,'example/PBMC/pbmc_ATAC_singlecellexperiment.rds')

### chromVAR running

In [ ]:
pbmc10x_chromVAR <- RunchromVAR(pbmc_10X_inputMat)

In [ ]:
# pbmc10x_chromVAR would have 3 keys: dev,variability,zscore
plotVariability(pbmc10x_chromVAR$variability, use_plotly = T) 

In [ ]:
qsave(pbmc10x_chromVAR, "10X_ATAC/per_50_cell_peak/1k_TF_index/run_chromVAR_result.qs")

In [ ]:
pbmc10x_chromVAR <- qread("10X_ATAC/per_50_cell_peak/1k_TF_index/run_chromVAR_result.qs")

In [ ]:
TFlist <- head(pbmc10x_chromVAR$variability[order(pbmc10x_chromVAR$variability['variability'], decreasing=T),],5)$name
z_frame <- t(pbmc10x_chromVAR$zscore)

In [ ]:
seurat_rds <- readRDS('10X_ATAC/analysis/atac_pbmc_10k_v1_S1_L001/Result/Analysis/atac_pbmc_10k_v1_S1_L001_scATAC_Object.rds')
chromVarResult <- mergeChromVarResult(seurat_rds, z_frame)

In [ ]:
chromVarResult$ATAC@meta.data

In [ ]:
for (i in TFlist) {
    fig <- FeaturePlot(object = chromVarResult$ATAC, feature=i)
    show(fig)
}

In [ ]:
# tmp.path <- "10X_ATAC/chromVAR_result_more_motif.rds"
# tmp <- readRDS(tmp.path)
# qsave(tmp, str_replace(tmp.path, ".rds", ".qs"))

In [ ]:
qsave(chromVarResult, "10X_ATAC/chromVAR_result_more_motif.qs")

In [ ]:
chromVarResult <- qread("example/10X_ATAC/chromVAR_result_more_motif.qs")

In [ ]:
write.table(chromVarResult$z,'example/10X_ATAC/chromVar_result_table.txt', quote=FALSE, sep='\t')